# Creación de nuevos tipos de datos en Julia

## ¿Por qué necesitamos crear nuevos tipos?

En el notebook 1, empezamos a desarrollar la aritmética de intervalos. Para hacerlo, tratamos a un intervalo como un par ordenado de números de la forma $(a, b)$, representando el intervalo cerrado $[a, b]$. Pero un vil par de números ¡*puede representar muchos tipos de objetos diferentes*! --por ejemplo, un número complejo, un vector de dos componentes, o un "número dual" (ver más tarde en el curso).
Además, un intervalo no es únicamente un par ordenado de números, sino cuenta con otras propiedades que queremos capturar.

Para poder representar dentro de la computadora la gama de propiedades de un intervalo, y para distinguirlo de otros objetos que se representen superficialmente de la misma manera, requerimos codificar la definición de un intervalo en un *nuevo tipo de datos*, que llamaremos `Intervalo`.

## Despacho múltiple ("multiple dispatch")

Una característica clave de Julia, que lo distingue de la gran mayoría de los demás lenguajes, es el "despacho múltiple". Esto quiere decir que una misma función (llamada una *función genérica*) puede tener varios *métodos* que actúen sobre objetos de diferentes tipos.

Pensemos en el notebook 1. Podríamos querer definir la función `f` actuando sobre una variable para representar la función usual $f: \mathbb{R} \to \mathbb{R}$, pero cuando actúa sobre dos variables, tratamos a estos dos variables como representando un intervalo, y entonces aplicamos la extensión intervalar $\tilde{f}: \mathbb{IR} \to \mathbb{IR}$ que mapea un intervalo a su rango.

**[1]** (i) Implementa la función $f(x) = x^2 - 2x$ de las dos formas, con `f(x)` y `f(a, b)`.

(ii) ¿Qué arroja `methods(f)`?

(iii) Escribe un método de `f` tratando al intervalo como un par ordenado ("tupla" / "tuple"). Para hacerlo, al momento de definir la función, se incluye una *anotación de tipo* ("type annotation") `::Tuple` o `::NTuple{2}` (para especificar que debe tener dos entradas):

    f(x::Tuple) = ...
    
(iv) Ahora ¿qué arroja `methods(f)`?`

## Creación de tipos

Julia nos permite crear / definir nuevos *tipos de datos*, o *tipos* ("types") para representar objetos en el programa que se comporten de cierta forma, por ejemplo para modelar un objeto en el mundo real, o un nuevo tipo de número. En Julia (a diferencia de muchos otros lenguajes), los tipos que definamos los usuarios tienen *exactamente el mismo "nivel" que los tipos pre-definidos*.

La definición de un tipo para representar un intervalo se ve así:

In [ ]:
struct Intervalo
    inf::Float64
    sup::Float64
end

Esto define un nuevo tipo de objeto que se llama `Intervalo`, y especifica los datos que contiene: dos `Float64`, con los nombre `inf` (cota inferior) y `sup` (cota superior). Es decir, podemos pensar que un objeto de tipo `Intervalo` corresponde a una "caja" con estos datos adentro. Podremos crear distintos objetos que tienen el mismo *tipo*, pero que difieren entre sí; corresponde a crear distintas cajas que se ven superficialmente iguales, pero tienen cosas diferentes adentro.

[Nota: Las definiciones de tipos no se pueden modificar dentro de una misma sesión de Julia.]

### Constructores


Hasta ahora, no se ha creado ningún objeto de este tipo; solo hemos especificado una plantilla para cómo se ven todo `Intervalo`. Para poder crear un objeto de este tipo, Julia nos provee, de forma automática (por el momento), una función con el mismo nombre que el tipo; esta función se llama el *constructor* del tipo.

**[2]** (i) Encuentra cómo utilizar el constructor para crear un intervalo $X = [3, 4]$.

(ii) Verifica que puedes extraer la información adentro del objeto utilizando `X.<algo>`. [Pista: Para ver las opciones, presione `TAB` después del `.`.]

(iii) ¿Se puede modificar los datos? ¿Por qué?

(iv) ¿Qué tipos se pueden utilizar como argumentos al constructor?

(v) La función `parse` se utiliza para convertir una cadena en un número. Utiliza la documentación de `parse` (el cual se obtiene con `?parse`) para escribir un nuevo *método* (versión) del constructor que acepta dos cadenas. [Pista: Se especifica que un argumento debe ser de tipo `String` con `::String`.]

(vi) Escribe unos tests sencillos del constructor de `Intervalo`. Utiliza `@test_throws` para verificar los errores que pueda arrojar el constructor.

## Sobrecarga de operadores

Ahora sabemos cómo definir una variables $X = [3, 4]$ y $Y = [5, 6]$. Pero las operaciones aritméticas, como las vimos en el notebook 1, son torpes: para dos intervalos debemos escribir `suma(X, Y)` en lugar de `X + Y`.

**[3]** Define la función `suma` para que funcione únicamente para dos `Intervalo`s, usando la misma regla que en el notebook 1.

Julia, como la mayoría de los lenguajes modernos, permite utilizar la *sobrecarga de operadores* para extender la definición de `+` a nuestro nuevo tipo.

Para extender una función que viene definida en `Base` (la parte básica del lenguaje), como lo es `+`, se tiene que importar:

In [ ]:
import Base: +, -

**[4]** (i) Examina la lista de *métodos* (versiones) que ya existen de la función `+` usando la función `methods`.

(ii) Agrega un método nuevo que sume dos intervalos. [Pista: Basta con definir `+` con dos argumentos especificados como de tipo `Intervalo`, como si fuera una función nueva.]

(iii) Agrega un método que sume un intervalo y un número real. El número se puede especificar con el tipo abstracto `Real`.

(iv) Agrega un método que sume un número real y un intervalo en el otro orden.

(v) Escribe tests para estos métodos.

**[5]** Escribe las funciones `-` y `*`, así como tests para ellos.

**[6]** (i) Escribe un método de `/` para dividir un intervalo por un número. ¿Cuáles son los casos especiales?

(ii) Para dividir dos intervalos `X` y `Y`, hay un caso particular. ¿Cuál es? ¿Cuál debe ser el resultado?

(iii) Escribe un método para la función `inv` que calcula la inversa de un intervalo `x`, es decir `1 / x`.

(iv) Utiliza la función `inv` para definir `X / Y` en general para dos intervalos `X` y `Y`.

(v) Escribe tests para `/`.

### Constructores internos

Podríamos decidir que es conveniente restringir la definición de intervalos $[a, b]$ a la situación en la cual $a \le b$. Por el momento no podemos hacer esto.

Para lograrlo, debemos "interferir" con el proceso de construcción de un tipo, permitiendo que se construya sólo si se cumpla la condición que requeramos. Para hacerlo, Julia provee *constructores internos* ("inner constructors").

**[7]** Busca documentación sobre constructores internos y utilízalos para impedir que se crean `Intervalo`s con $a > b$. 

**[8]** ¿Qué interpretación podría tener un interval $[a, b]$ con $a > b$? (Hay distintas respuestas posibles.)

## Resumen

**[9]** Escribe un resumen de lo que hayamos visto en este notebook.